# Paint with Perceptron
A Perceptron trained with the MNIST dataset will be able to recognize pictures with digits. 
Using it, I want to create a Paint, which can recognize digits.

Used: 
- MNIST dataset
- Tensorflow 
- JupyterLab
- Pandas
- Matplotlib
- Seaborn

Import libs:

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import layers

import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

tf.version.VERSION

'2.15.0'

## Load and split dataset
Load MNIST dataset from *tensorflow-datasets*. Split it and shuffle the data. 
`as_supervised=True` for get a tuple insted dictionary.

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
);

## Build a training pipeline

Apply the following transformations:

- `tf.data.Dataset.map`: TFDS provide images of type `tf.uint8`, while the model expects `tf.float32`. Therefore, I need to normalize images.
- `tf.data.Dataset.cache`: Cache it before shuffling for a better performance.
- `tf.data.Dataset.shuffle`: For true randomness, set the shuffle buffer to the full dataset size.
- `tf.data.Dataset.batch`: Batch elements of the dataset after shuffling to get unique batches at each epoch.
- `tf.data.Dataset.prefetch`: The end of transformations pipline.

In [3]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label


ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

## Build a test pipeline

Test transformation pipline is similar to the training once, but with small difference.

- Don't need to use `shuffle`
- Caching is done after batching.

In [4]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)